In [1]:
import pandas as pd
import sklearn
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn import model_selection, naive_bayes, svm, metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score


In [2]:
df = pd.read_csv('reports_with_vectors_1000.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112095 entries, 0 to 112094
Columns: 1009 entries, Unnamed: 0 to you
dtypes: float64(1005), int64(1), object(3)
memory usage: 862.9+ MB


In [4]:
df.head()

,Unnamed: 0,shape,city_latitude,city_longitude,year,month,day,hour,minute,duration_parsed,...,working,world,would,yard,yards,year.1,years,yellow,yet,you
0,0,circle,44.941247,-123.004235,NaN,NaN,NaN,NaN,NaN,900.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0
1,1,oval,44.651300,-68.450700,NaN,NaN,NaN,NaN,NaN,300.0,...,0.0,0.0,0.0,0.0,0.0,0.11174,0.092858,0.0,0.0,0.0
2,2,light,29.488866,-98.475240,2017.0,1.0,25.0,20.0,45.0,5400.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0
3,3,circle,27.285686,-80.363444,2017.0,2.0,24.0,20.0,45.0,300.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0
4,4,light,48.411100,-114.337600,2017.0,2.0,22.0,21.0,0.0,900.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0


In [5]:
df['shape'].value_counts()

light        22762
circle       18504
sphere       16016
other        15607
triangle     10497
oval          7050
cylinder      4404
formation     3456
changing      2716
flash         1963
rectangle     1848
diamond       1650
chevron       1294
cone           468
cross          344
Name: shape, dtype: int64

In [6]:
df = df[pd.notnull(df['shape'])]

In [7]:
df = df.dropna()

In [8]:
cat_cols = ['region']

for col in cat_cols:
    temp = pd.get_dummies(df[col], prefix=col)
    df= pd.concat([df, temp], axis=1)

In [9]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
y= Encoder.fit_transform(df['shape'])

In [10]:
x = df.drop(columns =['shape', 'Unnamed: 0','citystate','region','city_latitude','city_longitude','duration_parsed'])

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=1,stratify=y)

In [12]:
naive_model = naive_bayes.MultinomialNB()
naive_model.fit(x_train,y_train)

nb_train_predictions =naive_model.predict(x_train)
nb_test_predictions= naive_model.predict(x_test)


train_probs_nb = naive_model.predict_proba(x_train)
test_probs_nb = naive_model.predict_proba(x_test)

In [13]:
accuracy_score(nb_train_predictions, y_train)

0.2906686626746507

In [14]:
accuracy_score(nb_test_predictions, y_test)

0.2878778984443792

In [15]:
from sklearn.metrics import log_loss
log_loss(y_train, train_probs_nb)

2.1522725942009817

In [16]:
log_loss(y_test, test_probs_nb)

2.1728219186945323

In [17]:
print(metrics.classification_report(y_test, nb_test_predictions))



              precision    recall  f1-score   support

           0       0.50      0.00      0.00       437
           1       0.33      0.00      0.01       214
           2       0.21      0.52      0.30      2770
           3       0.00      0.00      0.00        70
           4       0.00      0.00      0.00        56
           5       0.67      0.11      0.20       664
           6       1.00      0.01      0.02       252
           7       0.15      0.04      0.07       316
           8       0.15      0.02      0.03       549
           9       0.30      0.49      0.37      3636
          10       0.24      0.15      0.18      2454
          11       0.82      0.02      0.04      1071
          12       0.83      0.02      0.03       291
          13       0.46      0.34      0.39      2552
          14       0.64      0.19      0.29      1703

    accuracy                           0.29     17035
   macro avg       0.42      0.13      0.13     17035
weighted avg       0.40   

C:\Users\Lucyc\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.5, 1, 1.5, 2]
}
# Instantiate the grid search model
nv_clf = GridSearchCV(estimator = naive_model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2).fit(x_train, y_train)

clf = nv_clf.best_estimator_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   45.7s remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   51.4s finished


In [19]:
clf.fit(x_train,y_train)
nb_train_predictions =clf.predict(x_train)
nb_test_predictions= clf.predict(x_test)

nb_train_pred_proba = clf.predict_proba(x_train)
nb_test_pred_proba = clf.predict_proba(x_test)

In [20]:
clf

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [21]:
accuracy_score(nb_train_predictions, y_train)

0.29147587178584006

In [22]:
accuracy_score(nb_test_predictions, y_test)

0.2892280598767244

In [23]:
from sklearn.metrics import log_loss
log_loss(y_train, nb_train_pred_proba )

2.140291275724055

In [24]:
log_loss(y_test, nb_test_pred_proba )

2.1634907176103337